In [1]:
# Import Statements
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Lambda
from keras.callbacks import LambdaCallback

import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
# Open the filtered Shakespeare data 
file = open('./data/shakespeare_filt_2.txt', 'r')
text = file.read()
file.close()

In [3]:
# Organize the text into sequences of length characters, starting at every nth character 
length = 41
n = 1
sequences = [] 

for i in range(length, len(text), n):
    seq = text[i-length:i+1]
    sequences.append(seq)

In [4]:
# Finding all the unique characters in seq_text
chars = sorted(list(set(text)))

# Creating a mapping for each unique char to an integer
mapping = dict((c, i) for i, c in enumerate(chars))
reverse_mapping = dict((i, c) for i, c in enumerate(chars))

# Size of the dictionary
vocab_size = len(mapping)

encoded_sequences = []
# For every line in sequences, encode the sequences using the mapping
for seq in sequences:
    encoded_seq = [mapping[char] for char in seq]
    encoded_sequences.append(encoded_seq)

In [5]:
# Split the sequences into X and y
encoded_sequences = array(encoded_sequences)
X, y = encoded_sequences[:,:-1], encoded_sequences[:,-1]

# Use to_categorical to one hot encode the sequences
encoded_sequences = [to_categorical(x, num_classes=vocab_size) for x in X]

# Use to_categorical to one hot encode y
X = array(encoded_sequences)
y = to_categorical(y, num_classes=vocab_size)

X.shape

(93441, 41, 36)

In [6]:
# Define RNN model with # LSTM units (units) and temperature (temp)
def generate_model(X, units, temp):
    model = Sequential()
    # LSTM 
    model.add(LSTM(units, input_shape=(X.shape[1], X.shape[2])))
    
    # Add Lambda layer with temperature parameter 
    model.add(Lambda(lambda x: x / temp))
    
    # Softmax layer 
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
   
    return model

In [14]:
# I ran this cell multiple times with different parameters and saved the models 
model = generate_model(X,155,2.020)
model.fit(X, y, epochs=50, verbose=2)
model.save('./data/model_1_155_2.020.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 155)               119040    
_________________________________________________________________
lambda_1 (Lambda)            (None, 155)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 36)                5616      
Total params: 124,656
Trainable params: 124,656
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
 - 95s - loss: 2.4592 - accuracy: 0.2968
Epoch 2/50
 - 86s - loss: 2.0959 - accuracy: 0.3801
Epoch 3/50
 - 93s - loss: 1.9726 - accuracy: 0.4095
Epoch 4/50
 - 89s - loss: 1.8881 - accuracy: 0.4326
Epoch 5/50
 - 90s - loss: 1.8247 - accuracy: 0.4495
Epoch 6/50
 - 88s - loss: 1.7763 - accuracy: 0.4605
Epoch 7/50
 - 88s - loss: 1.7356 - accuracy: 0.4709
Epoch 8/50
 - 

In [7]:
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# Generate a sequence of words given a seed
def generate_seq(model, mapping, reverse_mapping, length, seed, n_chars):
    text = seed
     
    for i in range(n_chars):
        # Encode the characters 
        encoded = [mapping[char] for char in text]
        
        # Fixes the length of the sequence to be max length 
        encoded = pad_sequences([encoded], maxlen=length, truncating='pre')
        
        # Hot one encode the sequence
        encoded = to_categorical(encoded, num_classes=len(mapping))
        
        # Predict the characters 
        prediction = model.predict_classes(encoded, verbose=0)
        
        # Map the index to a character using reverse_mapping
        out_char = reverse_mapping[prediction[0]]
        text += out_char
        
    return text 

In [8]:
model = load_model('./data/model_4_150_1.h5')
print(model.summary())
print(generate_seq(model, mapping, reverse_mapping, 41, "shall i compare thee to a summer's day?\n", 1000))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 150)               112200    
_________________________________________________________________
lambda_1 (Lambda)            (None, 150)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 36)                5436      
Total params: 117,636
Trainable params: 117,636
Non-trainable params: 0
_________________________________________________________________
None
shall i compare thee to a summer's day?
o his coorule ill but that should i le press.
andeare thou make those didsicchestee even.
for strat apon you wear agt of briends same,
now forter oubless stall and mo gove willoss
by ghard, and moren enderted theidespreed,
which which it by strang time inas it be
against pface thee alt the world waskemed,
with a partyore dospiny, a

In [9]:
model = load_model('./data/model_2_100_1.h5')
print(model.summary())
print(generate_seq(model, mapping, reverse_mapping, 41, "shall i compare thee to a summer's day?\n", 1000))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 150)               112200    
_________________________________________________________________
lambda_2 (Lambda)            (None, 150)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 36)                5436      
Total params: 117,636
Trainable params: 117,636
Non-trainable params: 0
_________________________________________________________________
None
shall i compare thee to a summer's day?
the reeves be their partion one time,
thou art the bare's fore thee i joy not,
the other with all withing of wormest lie,
me is sight thee to foull far fre-how,
the one than in me despyest with self bridg
as with the cen ofe though i praise the brow,
then will for my love, that to worth in ackil,
a though i have so shall i am from 

In [10]:
model = load_model('./data/model_1_100_1.h5')
print(model.summary())
print(generate_seq(model, mapping, reverse_mapping, 41, "shall i compare thee to a summer's day?\n", 1000))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100)               54800     
_________________________________________________________________
lambda_4 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 36)                3636      
Total params: 58,436
Trainable params: 58,436
Non-trainable params: 0
_________________________________________________________________
None
shall i compare thee to a summer's day?
thy will not me thou be receiv'st of my more,
that i have sweet fair this sweet upon thee,
do works that which i constanter's shape,
the world not so thou be received and thee
were his black sweet fair with his charace
but thou dost thou art swears of thy self,
and thou shall summord of their fair frows,
and so not love hath thoughts

In [11]:
model = load_model('./data/model_1_100_1.50.h5')
print(model.summary())
print(generate_seq(model, mapping, reverse_mapping, 41, "shall i compare thee to a summer's day?\n", 1000))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 100)               54800     
_________________________________________________________________
lambda_5 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 36)                3636      
Total params: 58,436
Trainable params: 58,436
Non-trainable params: 0
_________________________________________________________________
None
shall i compare thee to a summer's day?
thou thy self and some thee that thou art,
and that i am thou art so fair praised than
thou art the world my love that beauty of me,
and that thou art the world my self to tell.
and the world of more that in the world,
and the strength in my self all the worth and love,
that i am thou art summer and the sum,
when i and beauty be thre

In [12]:
model = load_model('./data/model_1_100_0.75.h5')
print(model.summary())
print(generate_seq(model, mapping, reverse_mapping, 41, "shall i compare thee to a summer's day?\n", 1000))

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100)               54800     
_________________________________________________________________
lambda_6 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 36)                3636      
Total params: 58,436
Trainable params: 58,436
Non-trainable params: 0
_________________________________________________________________
None
shall i compare thee to a summer's day?
whence sweet self thou art as the seen thee,
who art as thou art forgot in truth desert,
and in the will every what i seen thee,
and in the will all-earies the strong,
for thou art forgot, than thou art as mine,
so that i have thought the day of thee in me,
whaten i am so thine the return of thy show,
the one doth that words which sh

In [13]:
model = load_model('./data/model_1_100_0.25.h5')
print(model.summary())
print(generate_seq(model, mapping, reverse_mapping, 41, "shall i compare thee to a summer's day?\n", 1000))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 100)               54800     
_________________________________________________________________
lambda_7 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 36)                3636      
Total params: 58,436
Trainable params: 58,436
Non-trainable params: 0
_________________________________________________________________
None
shall i compare thee to a summer's day?
thou are you with me ablacked that love told
still to hear that which i think of thine,
and such a breath, and to my dear pleasure.
so thee, thou dost not be thy dear fair render,
whilst thou art forgot, where beauty being,
and to the resto disgrace,
but when i summer's breathers beauty maken
which works my love to store summer of y

In [15]:
model = load_model('./data/model_1_155_2.020.h5')
print(model.summary())
print(generate_seq(model, mapping, reverse_mapping, 41, "shall i compare thee to a summer's day?\n", 1000))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 155)               119040    
_________________________________________________________________
lambda_1 (Lambda)            (None, 155)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 36)                5616      
Total params: 124,656
Trainable params: 124,656
Non-trainable params: 0
_________________________________________________________________
None
shall i compare thee to a summer's day?
thou art the proud charve lived that they deeds,
and the world make the world will i must,
but that i age of the world my self dissiged,
and that i an the world my self dothed,
and that thou art a somethered thee,
and i am the world may still that thou art,
and that i am the world may strange, the strange,
the worst that thou art a